# [모의 경진대회] 토지피복지도 객체 분할

* 이미지 세그멘테이션
* 담당: 박성호 M

## 데이터 디렉토리 구조

In [6]:
# DATA/
#   \_train/
#        \_traindf.csv  
#        \_images/
#            \_xxx.png
#            \_yyy.png
#            \_zzz.png
#            \_...  
#        \_masks/
#            \_xxx.png
#            \_yyy.png
#            \_zzz.png
#            \_...
#   \_test/
#        \_sample_submission.csv
#        \_testdf.csv
#        \_images/
#            \_aaa.png  
#            \_bbb.png  
#            \_...  

## 필수 라이브러리 불러오기

In [7]:
!pip install segmentation_models_pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 337.0 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 1.2 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.0/377.0 kB 940.4 kB/s eta 0:00:0000:0100:01
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=65cfe6dec777c83d634e810d174704b70304af503ea0183804bc6feffbe563a5
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60965 sha256=ffa767b9b47c70861b1c2233661ed5c39c83c8c0739f1d4c257d6e68435045fc
  Stored in directory: /root/.cache/pip/wheels/ed/27/e8/9543d42de2740d3544db96aefef63bda3f2c1761b3334f4873
Successfully built efficientnet-pytorch pretrainedmodels


In [8]:
import os
import numpy as np
import pandas as pd
import random

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm
from sklearn.model_selection import train_test_split

import segmentation_models_pytorch as smp
from segmentation_models_pytorch.losses import DiceLoss, SoftBCEWithLogitsLoss, SoftCrossEntropyLoss

import albumentations as A
import cv2
from datetime import datetime, timezone, timedelta

## 하이퍼파라미터 및 기타 인자 설정

#### 데이터 경로

In [9]:
# 프로젝트 경로
PROJECT_DIR = '/kaggle/input/landmap-dataset/'
os.chdir(PROJECT_DIR)

#데이터 경로
DATA_DIR = os.path.join(PROJECT_DIR) # 모든 데이터가 들어있는 폴더 경로
TRAIN_DIR = os.path.join(DATA_DIR, 'train') # 학습 데이터가 들어있는 폴더 경로
TRAIN_IMG_DIR = os.path.join(TRAIN_DIR, 'images') # 학습 이미지가 들어있는 폴더 경로
TRAIN_MASK_DIR = os.path.join(TRAIN_DIR, 'masks') # 학습 마스크가 들어있는 폴더 경로
TRAIN_CSV_FILE = os.path.join(TRAIN_DIR, 'traindf.csv') # 학습 이미지와 마스크 이름이 들어있는 CSV 경로

### 데이터 수량 확인:
- n_train = 3930
- n_test = 3930

In [10]:
len(os.listdir(TRAIN_IMG_DIR)) #3930

3930

In [11]:
len(os.listdir(TRAIN_MASK_DIR)) #3930

3930

### 결과 저장 경로 설정

In [12]:
# 시간 고유값 
kst = timezone(timedelta(hours=9))        
train_serial = datetime.now(tz=kst).strftime("%Y%m%d_%H%M%S")

# 기록 경로
RECORDER_DIR = os.path.join('/kaggle/working/', 'results', 'train', train_serial)
# 현재 시간 기준 폴더 생성
os.makedirs(RECORDER_DIR, exist_ok=True)    

#### 시드 설정

In [13]:
RANDOM_SEED = 2022 #랜덤 시드

torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

#### 디바이스 설정

In [14]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#### 하이퍼파라미터 설정

In [15]:
EPOCHS = 50
BATCH_SIZE = 8 #8 # 줄여보자
LEARNING_RATE = 0.003
EARLY_STOPPING_PATIENCE = 10
IMG_SIZE = 512

ENCODER = 'timm-efficientnet-b4'# 활용할 인코더 모델
WEIGHTS = 'imagenet' # Pre-train에 활용된 데이터셋

## Dataset 정의

In [16]:
class SegDataset(Dataset):
    def __init__(self, df, augmentations, img_dir, mask_dir):
        self.df = df # 이미지와 마스크 이름이 저장된 데이터프레임 
        self.augmentations = augmentations # 학습 전 적용할 augmentation
        self.img_dir = img_dir # 이미지 폴더 경로
        self.mask_dir = mask_dir # 마스크 폴더 경로
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        # 데이터 프레임 불러와서 이미지와 마스크 경로 설정
        row = self.df.iloc[idx] # 데이터프레임 행 불러오기
        image_path = os.path.join(self.img_dir,row['img'])
        mask_path = os.path.join(self.mask_dir, row['mask'])
        
        # 이미지와 마스크 불러오기
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        mask = np.expand_dims(mask, axis=-1)
        
        # Augmentation 적용하기
        if self.augmentations:
            data = self.augmentations(image=image, mask=mask)
            image = data['image']
            mask = data['mask']
        
        # PyTorch 인풋 모양에 맞게 이미지와 마스크 모양 변경
        image = np.transpose(image, (2,0,1)).astype(np.float32)
        mask = np.transpose(mask, (2,0,1)).astype(np.float32)
        
        # 이미지 Normalization 0~255 픽셀값 --> 0~1 픽셀값
        image = torch.Tensor(image) / 255.0
        mask = torch.round(torch.Tensor(mask)/255.0)
        
        return image, mask

## 모델 정의

In [17]:
class SegModel(nn.Module):
    def __init__(self):
        super(SegModel, self).__init__()
        
        # Pre-train된 UNET 불러오기
        self.backbone = smp.Unet(
            encoder_name = ENCODER, # 인코더 모델 설정
            encoder_weights = WEIGHTS, # 사전학습 데이터셋 설정
            in_channels = 3, # 이미지 디멘션 (3 * 512 * 512)
            classes = 1, # 세그멘테이션 클래스 개수 
            activation = None # logit 값 불러오기
        )
        
    def forward(self, images):
        logits = self.backbone(images)
        
        return logits

## Utils 정의
#### Augmentation 함수

In [18]:
# def get_train_augs():
#     return A.Compose([
#         A.Resize(IMG_SIZE, IMG_SIZE), # 이미지 크기 변환
#         A.HorizontalFlip(p=0.5), # 이미지 좌우반전
#         A.VerticalFlip(p=0.5) # 이미지 상하반전
#     ])

# def get_valid_augs():
#     return A.Compose([
#         A.Resize(IMG_SIZE, IMG_SIZE)
#     ])

In [19]:
def get_train_augs():
    return A.Compose([
        A.RandomResizedCrop(height=IMG_SIZE, width=IMG_SIZE, p=1.0),
        A.Rotate(20),
        A.Flip(),
        A.Transpose(),
    A.Resize(height=IMG_SIZE, width=IMG_SIZE, p=1.0),
    A.Normalize(p=1.0),
    ], p=1.0)

def get_valid_augs():
    return A.Compose([
    A.Resize(height=IMG_SIZE, width=IMG_SIZE, p=1.0),
    A.Normalize(p=1.0),
])

#### Train 함수

In [20]:
def train_fn(dataloader, model, optimizer, loss_fn):
    model.train()
    
    total_loss = 0.0
    
    for images,masks in tqdm(dataloader):
        images = images.to(DEVICE)
        masks = masks.to(DEVICE)
        
        optimizer.zero_grad()
        logits = model(images)
        loss = loss_fn(logits, masks)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
    return total_loss/len(dataloader)

#### Validation 함수

In [21]:
def valid_fn(dataloader, model, loss_fn):
    model.eval()
    
    total_loss = 0.0
    
    with torch.no_grad():
        for images,masks in tqdm(dataloader):
            images = images.to(DEVICE)
            masks = masks.to(DEVICE)
            logits = model(images)
            loss = loss_fn(logits, masks)
            total_loss += loss.item()
    return total_loss/len(dataloader)

## 모델 학습
#### Dataset & Dataloader 설정

In [22]:
# 학습 이미지, 마스크 이름 들어있는 CSV 불러와 데이터 프레임으로 저장
entiredf = pd.read_csv(TRAIN_CSV_FILE)

# Train과 Validation 데이터셋으로 나누기
traindf, validdf = train_test_split(entiredf, test_size=0.2)
traindf = traindf.reset_index(drop=True)
validdf = validdf.reset_index(drop=True)

# Dataset 및 Dataloader 설정
train_dataset = SegDataset(traindf, get_train_augs(), TRAIN_IMG_DIR, TRAIN_MASK_DIR)
valid_dataset = SegDataset(validdf, get_valid_augs(), TRAIN_IMG_DIR, TRAIN_MASK_DIR)
train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size = BATCH_SIZE)

#### 모델과 기타 utils 설정

In [23]:
class ComboBCEDiceLoss(nn.Module):
    """
        Combination BinaryCrossEntropy (BCE) and Dice Loss with an optional running mean and loss weighing.
    """

    def __init__(self, use_running_mean=False, bce_weight=1, dice_weight=1, eps=1e-6, gamma=0.9, combined_loss_only=True, **_):
        """
        :param use_running_mean: - bool (default: False) Whether to accumulate a running mean and add it to the loss with (1-gamma)
        :param bce_weight: - float (default: 1.0) Weight multiplier for the BCE loss (relative to dice)
        :param dice_weight: - float (default: 1.0) Weight multiplier for the Dice loss (relative to BCE)
        :param eps: -
        :param gamma:
        :param combined_loss_only: - bool (default: True) whether to return a single combined loss or three separate losses
        """

        super().__init__()
        '''
        Note: BCEWithLogitsLoss already performs a torch.sigmoid(pred)
        before applying BCE!
        '''
        self.bce_logits_loss = nn.BCEWithLogitsLoss()

        self.dice_weight = dice_weight
        self.bce_weight = bce_weight
        self.eps = eps
        self.gamma = gamma
        self.combined_loss_only = combined_loss_only

        self.use_running_mean = use_running_mean
        self.bce_weight = bce_weight
        self.dice_weight = dice_weight

        if self.use_running_mean is True:
            self.register_buffer('running_bce_loss', torch.zeros(1))
            self.register_buffer('running_dice_loss', torch.zeros(1))
            self.reset_parameters()

    def to(self, device):
        super().to(device=device)
        self.bce_logits_loss.to(device=device)

    def reset_parameters(self):
        self.running_bce_loss.zero_()
        self.running_dice_loss.zero_()

    def forward(self, outputs, labels, **_):
        # inputs and targets are assumed to be BxCxWxH (batch, color, width, height)
        outputs = outputs.squeeze()       # necessary in case we're dealing with binary segmentation (color dim of 1)
        if len(outputs.shape) != len(labels.shape):
            raise AssertionError
        # assert that B, W and H are the same
        if outputs.size(-0) != labels.size(-0):
            raise AssertionError
        if outputs.size(-1) != labels.size(-1):
            raise AssertionError
        if outputs.size(-2) != labels.size(-2):
            raise AssertionError

        bce_loss = self.bce_logits_loss(outputs, labels)

        dice_target = (labels == 1).float()
        dice_output = torch.sigmoid(outputs)
        intersection = (dice_output * dice_target).sum()
        union = dice_output.sum() + dice_target.sum() + self.eps
        dice_loss = (-torch.log(2 * intersection / union))

        if self.use_running_mean is False:
            bmw = self.bce_weight
            dmw = self.dice_weight
            # loss += torch.clamp(1 - torch.log(2 * intersection / union),0,100)  * self.dice_weight
        else:
            self.running_bce_loss = self.running_bce_loss * self.gamma + bce_loss.data * (1 - self.gamma)
            self.running_dice_loss = self.running_dice_loss * self.gamma + dice_loss.data * (1 - self.gamma)

            bm = float(self.running_bce_loss)
            dm = float(self.running_dice_loss)

            bmw = 1 - bm / (bm + dm)
            dmw = 1 - dm / (bm + dm)

        loss = bce_loss * bmw + dice_loss * dmw

        if self.combined_loss_only:
            return loss
        else:
            return loss, bce_loss, dice_loss

In [24]:
#PyTorch
ALPHA = 0.5 # < 0.5 penalises FP more, > 0.5 penalises FN more
BETA = 0.5
CE_RATIO = 0.5 #weighted contribution of modified CE loss compared to Dice loss

class ComboLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(ComboLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1, alpha=ALPHA, beta=BETA, eps=1e-9):
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)

        #True Positives, False Positives & False Negatives
        intersection = (inputs * targets).sum()    
        dice = (2. * intersection + smooth) / (inputs.sum() + targets.sum() + smooth)

        inputs = torch.clamp(inputs, eps, 1.0 - eps)       
        out = - (ALPHA * ((targets * torch.log(inputs)) + ((1 - ALPHA) * (1.0 - targets) * torch.log(1.0 - inputs))))
        weighted_ce = out.mean(-1)
        combo = (CE_RATIO * weighted_ce) - ((1 - CE_RATIO) * dice)

        return combo

In [25]:
model = SegModel().to(DEVICE) # 모델 설정
loss_fn = DiceLoss(mode = 'binary') # 학습 loss funciton 설정   
# loss_fn = SoftBCEWithLogitsLoss()
# loss_fn = SoftCrossEntropyLoss()
# loss_fn = ComboBCEDiceLoss()  

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE) # optimizer 설정
# optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9)

# lr_scheduler = optim.lr_scheduler.MultiStepLR(optimizer=optimizer, milestones=[int(EPOCHS * 0.5), int(EPOCHS * 0.75)], gamma=0.1, last_epoch=-1)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/tf_efficientnet_b4_aa-818f208c.pth" to /root/.cache/torch/hub/checkpoints/tf_efficientnet_b4_aa-818f208c.pth


  0%|          | 0.00/74.4M [00:00<?, ?B/s]

#### Epoch 단위 학습 진행

In [26]:
best_loss = np.Inf

for i in range(EPOCHS):
    train_loss = train_fn(train_loader, model, optimizer, loss_fn)
    valid_loss = valid_fn(valid_loader, model, loss_fn)
    
    # loss가 감소하면 모델 저장
    if valid_loss < best_loss:
        torch.save(model.state_dict(), os.path.join(RECORDER_DIR, "best-model.pt"))
        print('saved model')
        best_loss = valid_loss
        print(f"Epoch: {i+1}, Train Loss: {train_loss} Valid Loss: {valid_loss}")

100%|██████████| 99/99 [00:31<00:00,  3.11it/s]


saved model
Epoch: 1, Train Loss: 0.34534670894079234 Valid Loss: 0.24856314936069526


100%|██████████| 99/99 [00:25<00:00,  3.82it/s]


saved model
Epoch: 2, Train Loss: 0.28454103436482164 Valid Loss: 0.21650766242634167


100%|██████████| 99/99 [00:27<00:00,  3.59it/s]


...

100%|██████████| 99/99 [00:26<00:00,  3.77it/s]


saved model
Epoch: 35, Train Loss: 0.17486182211616264 Valid Loss: 0.15040171567839805


100%|██████████| 99/99 [00:26<00:00,  3.70it/s]


saved model
Epoch: 49, Train Loss: 0.16029762478578485 Valid Loss: 0.1495883446751219


100%|██████████| 99/99 [00:26<00:00,  3.70it/s]


### 성능 지표
resnet50 
- 성능 안좋음

effnet b4 (epoch 50, model 40), so far best model
- Epoch: 20, Train Loss: 0.13257013098277512 Valid Loss: 0.13606521095892396
- Epoch: 25, Train Loss: 0.1300631359636632 Valid Loss: 0.13446238065006758
- Epoch: 40, Train Loss: 0.12472128397939163 Valid Loss: 0.12788941342421253
- Epoch: 48, Train Loss: 0.11823700175030541 Valid Loss: 0.12040891189767856

vgg19 (epoch 20)
- Epoch: 19, Train Loss: 0.18212765864743533 Valid Loss: 0.17727654028420498

mobilenet_v2
- Epoch: 13, Train Loss: 0.18104453669249557 Valid Loss: 0.1695225786681127
- Epoch: 21, Train Loss: 0.1643344499075989 Valid Loss: 0.16186977516521106
- Epoch: 26, Train Loss: 0.1639832902803979 Valid Loss: 0.16000752918647998
- Epoch: 27, Train Loss: 0.1583945419042165 Valid Loss: 0.15862789719995826

effenet b8
- processing...



## 추론

#### 마스크를 RLE 형태로 변환해주는 함수

In [27]:
def mask_to_rle(mask):
    flatten_mask = mask.flatten()
    if flatten_mask.max() == 0:
        return f'0 {len(flatten_mask)}'
    idx = np.where(flatten_mask!=0)[0]
    steps = idx[1:]-idx[:-1]
    new_coord = []
    step_idx = np.where(np.array(steps)!=1)[0]
    start = np.append(idx[0], idx[step_idx+1])
    end = np.append(idx[step_idx], idx[-1])
    length = end - start + 1
    for i in range(len(start)):
        new_coord.append(start[i])
        new_coord.append(length[i])
    new_coord_str = ' '.join(map(str, new_coord))
    return new_coord_str

#### Test 데이터셋 불러오기

In [28]:
class TestDataset(Dataset):
    def __init__(self, df, img_dir):
        self.df = df
        self.img_dir = img_dir
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        imname = row['img']
        image_path = os.path.join(self.img_dir,imname)
        
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = np.transpose(image, (2,0,1)).astype(np.float32)
        image = torch.Tensor(image) / 255.0
        
        return image,imname

#### 경로 및 기타 인자 설정

In [29]:
TEST_DIR = os.path.join(DATA_DIR, 'test') # 테스트 데이터가 들어있는 폴더 경로
TEST_IMG_DIR = os.path.join(TEST_DIR, 'images') # 테스트 이미지가 들어있는 폴더 경로
TEST_CSV_FILE = os.path.join(TEST_DIR, 'testdf.csv') # 테스트 이미지 이름이 들어있는 CSV 경로

#### 테스트 Dataset, DataLoader 설정

In [30]:
testdf = pd.read_csv(TEST_CSV_FILE)
test_dataset = TestDataset(testdf, TEST_IMG_DIR)
test_loader = DataLoader(dataset=test_dataset, batch_size=1,shuffle=False)

#### 최고 성능 모델 불러오기

In [31]:
model.load_state_dict(torch.load(os.path.join(RECORDER_DIR, 'best-model.pt')))

<All keys matched successfully>

#### 추론 진행

In [ ]:
file_list = [] # 이미지 이름 저장할 리스트
pred_list = [] # 마스크 저장할 리스트
class_list = [] # 클래스 이름 저장할 리스트 ('building')

model.eval()
with torch.no_grad():
    for batch_index, (image,imname) in tqdm(enumerate(test_loader)):
        image = image.to(DEVICE)
        logit_mask = model(image)
        pred_mask = torch.sigmoid(logit_mask) # logit 값을 probability score로 변경
        pred_mask = (pred_mask > 0.5) * 1.0 # 0.5 이상 확률 가진 픽셀값 1로 변환
        pred_rle = mask_to_rle(pred_mask.detach().cpu().squeeze(0)) # 마스크를 RLE 형태로 변경
        pred_list.append(pred_rle)
        file_list.append(imname[0])
        class_list.append("building")
        

1202it [01:21, 15.25it/s]

#### 예측 결과 파일 만들기

In [ ]:
# 예측 결과 데이터프레임 만들기
results = pd.DataFrame({'img_id':file_list,'class':class_list,'prediction':pred_list})

# sample_submission.csv와 같은 형태로 변형
sampledf = pd.read_csv(os.path.join(TEST_DIR, 'sample_submission.csv'))
sorter = list(sampledf['img_id'])
results = results.set_index('img_id')
results = results.loc[sorter].reset_index()
                       
# 결과 저장
results.to_csv(os.path.join(RECORDER_DIR, 'prediction.csv'), index=False)